Loading model.

In [1]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K

In [3]:
def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

In [4]:
smooth = 1e-15

In [5]:
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

In [6]:
def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

In [7]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import cv2
from glob import glob
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision

In [8]:
def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    return x

In [9]:
def read_mask(path1, path2):
    x1 = cv2.imread(path1, cv2.IMREAD_GRAYSCALE)
    x2 = cv2.imread(path2, cv2.IMREAD_GRAYSCALE)
    x = x1 + x2
    x = cv2.resize(x, (W, H))
    x = x/np.max(x)
    x = x > 0.5
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)
    return x

In [10]:
from tqdm import tqdm
from tensorflow.keras.utils import CustomObjectScope

In [11]:
metrics = [dice_coef, iou, Recall(), Precision()]

In [12]:
with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef, 'dice_loss': dice_loss}):
  model = tf.keras.models.load_model("/content/drive/MyDrive/files/model.h5")

In [13]:
x = "/content/drive/MyDrive/MontgomerySet/CXR_png/MCUCXR_0001_0.png"
""" Extracing the image name. """
image_name = x.split("/")[-1]

In [14]:
""" Reading the image """
ori_x = cv2.imread(x, cv2.IMREAD_COLOR)
ori_x = cv2.resize(ori_x, (512, 512))
x = ori_x/255.0
x = x.astype(np.float32)
x = np.expand_dims(x, axis=0)

In [15]:
y1 = "/content/drive/MyDrive/MontgomerySet/ManualMask/leftMask/MCUCXR_0001_0.png"
y2 = "/content/drive/MyDrive/MontgomerySet/ManualMask/rightMask/MCUCXR_0001_0.png"

""" Reading the mask """
ori_y1 = cv2.imread(y1, cv2.IMREAD_GRAYSCALE)
ori_y2 = cv2.imread(y2, cv2.IMREAD_GRAYSCALE)
ori_y = ori_y1 + ori_y2
ori_y = cv2.resize(ori_y, (512, 512))
ori_y = np.expand_dims(ori_y, axis=-1)  ## (512, 512, 1)
ori_y = np.concatenate([ori_y, ori_y, ori_y], axis=-1)  ## (512, 512, 3)

In [16]:
""" Predicting the mask. """
y_pred = model.predict(x)[0] > 0.5
y_pred = y_pred.astype(np.int32)

In [17]:
y_pred

array([[[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]],

       [[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]],

       [[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]],

       ...,

       [[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]],

       [[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]],

       [[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]]], dtype=int32)

In [18]:
""" Saving the predicted mask along with the image and GT """
save_image_path = f"/content/drive/MyDrive/results1/{image_name}"
y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1)

In [19]:
y_pred

array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=int32)

In [21]:
sep_line = np.ones((512, 10, 3)) * 255

In [22]:
cat_image = np.concatenate([ori_x, sep_line, ori_y, sep_line, y_pred*255], axis=1)
cv2.imwrite(save_image_path, cat_image)

True